In [1]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"
from psfutil_1d import *
from scipy.special import jv

from scipy.interpolate import interp1d
from scipy.optimize import fsolve
from matplotlib.patches import Circle

In [2]:
def psf_gaussian_2d(sigma: float) -> np.ndarray:
    """Generates a 2D Gaussian PSF."""
    x, y = np.meshgrid(np.arange(-NTOT//2, NTOT//2) / (sigma*SAMP),
                       np.arange(-NTOT//2, NTOT//2) / (sigma*SAMP))
    return np.exp(-0.5 * (np.square(x) + np.square(y)))\
            / (2.0*np.pi * sigma**2)

def psf_simple_airy(ldp: float, obsc: float = 0.31) -> np.ndarray:
    """Generates a simple Airy PSF."""
    r = np.hypot(*np.meshgrid(np.arange(-NTOT//2, NTOT//2),
                              np.arange(-NTOT//2, NTOT//2))) / (ldp*SAMP)
    return np.square(             jv(0, np.pi*r)      + jv(2, np.pi*r)
                     - obsc**2 * (jv(0, np.pi*r*obsc) + jv(2, np.pi*r*obsc))) \
           / (4.0*ldp**2 * (1-obsc**2)) * np.pi

def pixelate_psf_2d(psf: np.ndarray) -> np.ndarray:
    """Pixelates a 2D Point Spread Function (PSF)."""
    k = np.linspace(0, 1, NTOT, endpoint=False)
    k[-(NTOT//2):] -= 1; k *= SAMP
    return np.fft.irfft2(np.fft.rfft2(psf) *\
        np.sinc(k[None, :NTOT//2+1]) * np.sinc(k[:, None]), s=(NTOT, NTOT))

In [3]:
BAND = "H158"

psf_in = psf_simple_airy(LDP[BAND])
psf_in_t = np.fft.rfft2(np.fft.ifftshift(psf_in))
psf_inp = pixelate_psf_2d(psf_in)
psf_inp_t = np.fft.rfft2(np.fft.ifftshift(psf_inp))

psf_out = psf_gaussian_2d(SIGMA[BAND] * 1.5)
psf_out_t = np.fft.rfft2(np.fft.ifftshift(psf_out))
weight_t = psf_out_t / psf_inp_t
weight_t[:, NPIX:] = 0; weight_t[NPIX:-NPIX+1, :] = 0; weight_t.imag = 0
weight = np.fft.ifftshift(np.fft.irfft2(weight_t, s=(NTOT, NTOT)))

/tmp/ipykernel_41750/2106691221.py:10: RuntimeWarning: invalid value encountered in divide
  weight_t = psf_out_t / psf_inp_t


In [4]:
# We have to undersample the weight field, so...
weightu = np.zeros_like(weight)
weightu[::SAMP, ::SAMP] = weight[::SAMP, ::SAMP] * SAMP**2
weightu_t = np.fft.rfft2(np.fft.ifftshift(weightu))

# Let's compute the as-realized output PSFs, oversampled and undersampled.
psf_outo = np.fft.ifftshift(np.fft.irfft2(weight_t * psf_inp_t, s=(NTOT, NTOT)))
psf_outu = np.fft.ifftshift(np.fft.irfft2(weightu_t * psf_inp_t, s=(NTOT, NTOT)))

square_norm(psf_outu-psf_out) / square_norm(psf_out)

6.99791135636633e-06

In [5]:
def visualize_psf_2d(fig: mpl.figure.Figure, ax: mpl.axes.Axes,
                     psf: np.ndarray, **kwargs) -> None:
    """Visualizes a 2D Point Spread Function (PSF)."""

    vmin = kwargs.pop("vmin", psf.min())
    vmax = kwargs.pop("vmax", psf.max())
    gain = max(abs(vmin), abs(vmax))
    im = ax.imshow(psf, cmap="RdYlBu_r", vmin=-gain, vmax=gain,
                   origin="lower", **kwargs)
    cbar = fig.colorbar(im, ax=ax); cbar.ax.set_ylim((vmin, vmax))

def get_psf_hwhm_2d(psf: np.ndarray) -> float:
    """Calculates the half width at half maximum (HWHM) of a 2D PSF."""
    f = interp1d(np.linspace(0, NPIX//2, NTOT//2, endpoint=False),
                 psf[NTOT//2, NTOT//2:])
    return fsolve(lambda r: f(r) - 0.5*psf.max(), 1.0)[0]

In [6]:
fig, axs = plt.subplots(2, 3, figsize=(10.8, 5.4))
slice_ = np.s_[SAMP*20:-SAMP*20+1, SAMP*20:-SAMP*20+1]
extent = (-12-1/SAMP/2, 12+1/SAMP/2, -12-1/SAMP/2, 12+1/SAMP/2)

for i, psf in enumerate([psf_in, psf_inp, psf_out]):
    ax = axs[0, i]
    visualize_psf_2d(fig, ax, np.log10(psf[slice_]), extent=extent)
    ax.add_patch(Circle((0, 0), get_psf_hwhm_2d(psf), fill=False,
                        edgecolor="k", linewidth=1, linestyle='--'))
axs[0, 0].set_title("Unpixelated Input PSF")
axs[0, 1].set_title("Pixelated Input PSF")
axs[0, 2].set_title("Target Output PSF")

for i in range(5):
    ax = axs[i//3, i%3]
    if i%3 == 0: ax.set_ylabel('$y$ [pixel]')
    else: ax.get_yaxis().set_ticklabels([])
    if i//3 == 1: ax.set_xlabel('$x$ [pixel]')
    else: ax.get_xaxis().set_ticklabels([])
    ax.set_xticks(np.arange(-10, 11, 5))
    ax.minorticks_on()

visualize_psf_2d(fig, axs[1, 0], weight[slice_] * SAMP**2, extent=extent)
fig, axs[1, 0].scatter(*np.mgrid[-12:13, -12:13], c="b", s=0.05, alpha=0.25)
axs[1, 0].set_title("Weight Field (Undersampled)")
visualize_psf_2d(fig, axs[1, 1], (psf_outu-psf_out)[slice_] * 1e4, extent=extent)
axs[1, 1].set_title(r"Output PSF Residual $\times 10^4$")

visualize_psf_2d(fig, axs[1, 2],
                 np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(psf_outu-psf_out)).real)
                 [NTOT//2-NPIX*4:NTOT//2+NPIX*4+1, NTOT//2-NPIX*4:NTOT//2+NPIX*4+1] / SAMP**2 * 1e4,
                 extent=(-4-1/NPIX/2, 4+1/NPIX/2, -4-1/NPIX/2, 4+1/NPIX/2))
axs[1, 2].set_title(r"Output PSF Residual FT $\times 10^4$")
axs[1, 2].set_xlabel(r"$u$ [cycle per pixel]")
axs[1, 2].set_ylabel(r"$v$ [cycle per pixel]")
axs[1, 2].set_xticks(np.arange(-4, 5, 2))
axs[1, 2].minorticks_on()

# fig.tight_layout()
# plt.show()
fig.savefig(f'plots/regrid_2d.pdf', bbox_inches='tight')
plt.close(fig)